# Orbital Elements to Cartesian Coordinates

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np
import matplotlib.pyplot as plt
import time

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
# from tf_utils import Identity
from r2b import VectorError
from orbital_element import make_data_orb_elt, make_dataset_elt_to_cfg, make_dataset_cfg_to_elt
from orbital_element import OrbitalElementToConfig, make_model_elt_to_cfg
from orbital_element import ConfigToOrbitalElement, make_model_cfg_to_elt
from orbital_element import MeanToEccentricAnomaly, MeanToTrueAnomaly

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [5]:
# Lightweight serialization
fname = '../data/r2b/orbital_element.pickle'
vartbl = load_vartbl(fname)

In [6]:
# Create small data set for orbital elements; dictionaries of numpy arrays
n = 100
a_min = 1.0
a_max = 2.0
e_max = 1.0
inc_max = np.pi/4.0
seed=42
elts, cart = make_data_orb_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, inc_max=inc_max, seed=seed)

In [7]:
# Create a tensorflow Dataset instance in both directions
batch_size = 64
ds_e2c = make_dataset_elt_to_cfg(n=n, a_min=a_min, a_max=a_max, e_max=e_max, 
                                 inc_max=inc_max, seed=seed, batch_size=batch_size)
ds_c2e = make_dataset_cfg_to_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, 
                                 inc_max=inc_max, seed=seed, batch_size=batch_size)

In [8]:
# Example batch
elts, cart = list(ds_e2c.take(1))[0]

# Unpack orbital elements
a = elts['a']
e = elts['e']
inc = elts['inc']
Omega = elts['Omega']
omega = elts['omega']
f = elts['f']
mu = elts['mu']

# Unpack cartesian coordinates
q = cart['q']
v = cart['v']

# Review shapes
print(f'Example batch sizes:')
print(f'a    = {a.shape}')
print(f'e    = {e.shape}')
print(f'inc  = {inc.shape}')
print(f'Omega= {Omega.shape}')
print(f'omega= {omega.shape}')
print(f'f    = {f.shape}')
print(f'mu   = {f.shape}')
print(f'q    = {q.shape}')
print(f'v    = {v.shape}')

Example batch sizes:
a    = (64,)
e    = (64,)
inc  = (64,)
Omega= (64,)
omega= (64,)
f    = (64,)
mu   = (64,)
q    = (64, 3)
v    = (64, 3)


In [ ]:
# Run the layer on the batch of orbital elements
inputs_e2c = (a, e, inc, Omega, omega, f, mu)
cart_rec = OrbitalElementToConfig()(inputs_e2c)

In [ ]:
# Create a model mapping orbital elements to cartesian coordinates
model_e2c = make_model_elt_to_cfg()

In [ ]:
# Inputs to compile this model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss')}

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0}

In [ ]:
# Compile the e2c model
model_e2c.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [ ]:
# Verify that model matches rebound
model_e2c.evaluate(ds_e2c)

In [ ]:
# Summary of the model mapping orbital elements to position
# model_e2c.summary()

In [ ]:
# Run the layer on the batch of orbital elements
qx = q[:,0]
qy = q[:,1]
qz = q[:,2]
vx = v[:,0]
vy = v[:,1]
vz = v[:,2]
inputs_c2e = (qx, qy, qz, vx, vy, vz, mu)
elt_rec = ConfigToOrbitalElement()(inputs_c2e)

In [ ]:
# Review shapes
print(f'Example batch sizes:')
print(f'qx   = {qx.shape}')
print(f'vx   = {vx.shape}')
print(f'mu   = {mu.shape}')

In [ ]:
# Create a model mapping cartesian coordinates to orbital elements
model_c2e = make_model_cfg_to_elt()

In [ ]:
# Inputs to compile the c2e model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'a': keras.losses.MeanSquaredError(),
        'e': keras.losses.MeanSquaredError(),
        'inc': keras.losses.MeanSquaredError(),
        'Omega': keras.losses.MeanSquaredError(),
        'omega': keras.losses.MeanSquaredError(),
        'f': keras.losses.MeanSquaredError(),
       }

metrics = None

loss_weights = {'a': 1.0,
                'e': 1.0,
                'inc': 1.0,
                'Omega': 1.0,
                'omega': 1.0,
                'f': 1.0}

In [ ]:
# model_c2e.summary()

In [ ]:
# Compile the c2e model
model_c2e.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [ ]:
model_c2e.evaluate(ds_c2e)

In [ ]:
a, e, inc, Omega, omega, f, M, N = elt_rec

In [ ]:
# Test the mean anomaly convervsion functions
E = MeanToEccentricAnomaly()((M, e))
f_rec = MeanToTrueAnomaly()((M, e))

# Compute the RMS error
f_err = f_rec - f
rms_err = np.sqrt(np.mean(f_err * f_err))
print(f'RMS error of true anomaly f from mean anomaly M:')
print(f'{rms_err:5.2E}')